In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
"""
Check to make sure we can run something on Payman's Machine  <---

Adding a good dataset <--
Possibly change initialization of model ()
(Minor) Saving model checkpoints <--
Ensuring that all 4 GPUS are used <-- (Paymans Machine Task)
Evaluation ()

Checkpointed model from training aligns with Evaluation-Optimized Model

"""

In [2]:
%cd drive/MyDrive/SysML-Project/draft-files-v1/model-scaffold/

/content/drive/MyDrive/SysML-Project/draft-files-v1/model-scaffold


In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
%pip install torch==2.0.1

  Using cached torch-2.0.1-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_nccl_cu11-2.14.3-py3-none-manylinux1_x86_64.whl (177.1 MB)
  Using cached triton-2.0.0-1-cp38-cp38-manylinux2014_x86_64.manylinux_

In [2]:
import os
import torch
print(torch.__version__)

2.0.1+cu117


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.device_count())

cuda
1


In [6]:
!pip -q install sentencepiece
!pip -q install datasets
!pip -q install fairscale
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00


In [16]:

from tokenizer import Tokenizer
from model import ModelArgs, Transformer
# from model_umar import ModelArgs, Transformer
import tqdm
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer("./tokenizer.model")
tokenizer.pad_token = tokenizer.eos_token

model_args = ModelArgs()
model_args.vocab_size = len(tokenizer)

In [8]:
!export RANK=0

In [9]:
from fairscale.nn.model_parallel import initialize_model_parallel
import torch.distributed as dist

def setup(rank, world_size):
    "Sets up the process group and configuration for PyTorch Distributed Data Parallelism"
    os.environ["MASTER_ADDR"] = 'localhost'
    os.environ["MASTER_PORT"] = "12355"

    # Initialize the process group
    if not dist.is_initialized():
        dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    "Cleans up the distributed environment"
    dist.destroy_process_group()

setup(0, 1)


# COMMENT IF INITIATED, OTHERWISE UNCOMMENT!!!!!
initialize_model_parallel(1)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


In [10]:
# Get Dataset.
from datasets import load_dataset
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]["text"]

# some random small dataset i found online. we should move away from this after testing.
data = load_dataset("NeelNanda/pile-10k", split="train")
dataset = CustomDataset(data)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [17]:
# Model Details
start_epoch = 0
EPOCHS = 2
# model = Transformer(model_args)
# model = torch.nn.DataParallel(Transformer(model_args))
model = torch.nn.parallel.DistributedDataParallel(Transformer(model_args))
criterion = torch.nn.CrossEntropyLoss()

In [18]:
## Re-initialize the learnable weights
import math
for name, param in model.named_parameters():
    if param.requires_grad:
        if param.dim() >= 2:
            torch.nn.init.kaiming_uniform_(param)
        else:
            # bias terms or 1D params, uniform init
            bound = 1 / math.sqrt(param.size(0))
            torch.nn.init.uniform_(param, -bound, bound)

In [19]:
# Move model to cuda
model = model.to(device)

In [20]:
# Declare optimizer with finalized model parameters (after parallel + initialization)
optimizer = torch.optim.AdamW(model.parameters(), lr=3.0E-4)

# New Section

In [21]:
# losses = []
log_every = 1
print("Number of batches:", len(data_loader))
model.train()
with torch.autograd.detect_anomaly():
    for epoch in range(EPOCHS):
        for c, batch in tqdm.tqdm(enumerate(data_loader)):
            # Batch size x Max Seq LEn
            sample = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=model_args.max_seq_len)["input_ids"].to(device)
            # target = sample.clone().to(device)
            target = sample.detach()

            # Batch size x Max Seq Len x Vocab Size
            optimizer.zero_grad()
            prediction = model(sample, 0)

            # Ensure swapping of axes
            loss = criterion(prediction.transpose(1, 2), target)
            # loss.requires_grad_(True)
            loss.backward()

            # losses.append(loss.item())
            if c % log_every == 0:
                print(f"Step: {c}, Loss: {loss.item():.4f}")
            optimizer.step()

            del loss, prediction, sample, target

<ipython-input-21-f1e334fabee2>:5: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


Number of batches: 313


1it [00:00,  1.43it/s]

Step: 0, Loss: 10.3741


2it [00:01,  1.66it/s]

...

Step: 114, Loss: 8.1504


116it [01:16,  1.70it/s]

Step: 115, Loss: 8.1132


116it [01:17,  1.50it/s]


KeyboardInterrupt: ignored

In [21]:
def get_size(tensor):
    memory_size = tensor.element_size() * tensor.numel()
    print(f"Tensor size on GPU: {memory_size / (1024 * 1024):.2f} MB")
get_size(sample)
get_size(target)

Tensor size on GPU: 0.06 MB
Tensor size on GPU: 0.06 MB


In [24]:
optimizer.zero_grad()